# Customer Profiling 

### Purpose: <br>
The purpose of this script is to profile customers based on their internet plan usage and free data bonus usage over a six month period. The profiling will help support revenue drive and marketing efforts of the company while providing a better understanding of customer behavior.

In [2]:
import pandas as pd
import datetime

In [8]:
df = pd.read_excel('Telecoms Data.xlsx')

In [9]:
print(df.head())

  Primary City  Account Number CUSTOMER_TYPE  Data Subscribed (GB)  \
0      Onitsha         1000001        Retail                125.66   
1      Onitsha         1000001        Retail                250.29   
2      Onitsha         1000001        Retail                323.42   
3      Onitsha         1000001        Retail                361.53   
4      Onitsha         1000001        Retail                364.62   

   Data Bonus (GB)  Bonus Used         YM  
0        56.688644    0.000000 2019-08-01  
1       119.598595    0.000000 2019-10-01  
2       100.723187  100.723187 2019-07-01  
3       270.656311  270.656311 2019-11-01  
4       110.301048  110.301048 2019-12-01  


In [11]:
df['YM'] = df['YM'].astype(str)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60168 entries, 0 to 60167
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Primary City          60168 non-null  object 
 1   Account Number        60168 non-null  int64  
 2   CUSTOMER_TYPE         60168 non-null  object 
 3   Data Subscribed (GB)  60168 non-null  float64
 4   Data Bonus (GB)       60168 non-null  float64
 5   Bonus Used            60168 non-null  float64
 6   YM                    60168 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 3.2+ MB


# Calculating Loyalty Score
Counting months with data purchased for each customer <l>

A customer has "Purchased Data" if "Data Subscribed > 0"

In [17]:
loyalty_counts = df[df['Data Subscribed (GB)'] > 0].groupby('Account Number')['YM'].nunique().reset_index()
loyalty_counts.rename(columns={'YM': 'months_with_purchase'}, inplace=True)

In [22]:
#Defining a function to assign loyalty scores
def assign_loyalty_score(months_with_purchase):
    if months_with_purchase == 6:
        return 20
    elif months_with_purchase == 5:
        return 15
    elif months_with_purchase == 4:
        return 10
    elif months_with_purchase < 4:
        return 5
    else:
        return 1


In [23]:
loyalty_counts['Loyalty_Score'] = loyalty_counts['months_with_purchase'].apply(assign_loyalty_score)

# Calculate usage score:

Count months where bonus was not used for each customer<l>
if 'Bonus_Used' == 0, this indicates that bonus wasnt used.


In [35]:
bonus_not_used_counts = df[df['Bonus Used'] == 0].groupby('Account Number')['YM'].nunique().reset_index()
bonus_not_used_counts.rename(columns={'YM': 'months_bonus_not_used'}, inplace=True)

In [36]:
#Get total unique months per customer
total_months_per_customer = df.groupby('Account Number')['YM'].nunique().reset_index()
total_months_per_customer.rename(columns={'YM': 'total_months'}, inplace= True)

In [38]:
bonus_usage_data = pd.merge(total_months_per_customer, bonus_not_used_counts, on='Account Number', how='left').fillna(0)
bonus_usage_data['months_bonus_not_used'] = bonus_usage_data['months_bonus_not_used'].astype(int)

In [39]:
#Defien function to assign bonus usage scores
def assign_bonus_usage_score(months_bonus_not_used, total_months):
    if months_bonus_not_used == total_months:
        return 20
    elif months_bonus_not_used == (total_months -1) and total_months == 6:
        return 15
    elif months_bonus_not_used == (total_months - 2) and total_months == 6:
        return 10
    elif months_bonus_not_used < (total_months - 2) and months_bonus_not_used > 0 and total_months == 6:
        return 5
    else:
        return 1

In [44]:
bonus_usage_data['bonus_usage_score'] = bonus_usage_data.apply(
    lambda row: assign_bonus_usage_score(row['months_bonus_not_used'], row['total_months']), axis=1
)

# Combine Scores and Calculate Final Percentage

In [48]:
#Merge loyalty and bonus usage scores
customer_scores = pd.merge(loyalty_counts[['Account Number', 'Loyalty_Score']],
                           bonus_usage_data[['Account Number', 'bonus_usage_score']],
                           on='Account Number',
                           how='outer').fillna(0)

In [50]:
#Calculate Total Score
customer_scores['Total_Score'] = customer_scores['Loyalty_Score'] + customer_scores['bonus_usage_score']

#Max Score = 40
MAX_SCORE = 40
customer_scores['Final_Score_Percentage'] = (customer_scores['Total_Score'] / MAX_SCORE) * 100

# Classify Customer into Tiers

In [55]:
def customer_tier(score_percentage):
    if score_percentage > 85:
        return 'Tier 1 Priority Service' #[cite:19]
    elif 75 <= score_percentage <=85:
        return 'Tier 2 Priority Service' #[cite:20]
    elif 60 < score_percentage < 75:
        return 'Tier 3 Priority Service + Incentives' #[cite:21]
    else:
        return 'Tier 4 Priority Service + Incentives' #[cite:22]

customer_scores['Customer_Tier'] = customer_scores['Final_Score_Percentage'].apply(customer_tier)

# Add Customer type(Corporate/Retail) to the final profiling

In [58]:
#get unique customer types for each account number 
customer_type_map = df[['Account Number', 'CUSTOMER_TYPE']].drop_duplicates()
customer_profiling = pd.merge(customer_scores, customer_type_map, on='Account Number', how='left')

# Display Results

In [59]:
print("\n Customer Profiling Results:")
print(customer_profiling.sort_values(by='Final_Score_Percentage', ascending=False).head(20))


 Customer Profiling Results:
      Account Number  Loyalty_Score  bonus_usage_score  Total_Score  \
2784         1002785           20.0                 20         40.0   
1335         1001336           20.0                 20         40.0   
1805         1001806           20.0                 20         40.0   
3638         1003639           20.0                 20         40.0   
3830         1003831           20.0                 20         40.0   
1995         1001996           20.0                 20         40.0   
3887         1003888           20.0                 20         40.0   
6808         1006809           20.0                 20         40.0   
2106         1002107           20.0                 20         40.0   
4106         1004107           20.0                 20         40.0   
4130         1004131           20.0                 20         40.0   
1083         1001084           20.0                 20         40.0   
1722         1001723           20.0            

In [62]:
print("\n Distribution of Customers by Tier:")
print(customer_profiling['Customer_Tier'].value_counts())


 Distribution of Customers by Tier:
Customer_Tier
Tier 3 Priority Service + Incentives    5461
Tier 4 Priority Service + Incentives    4162
Tier 2 Priority Service                  219
Tier 1 Priority Service                  105
Name: count, dtype: int64


In [63]:
# SAVE AS CSV
customer_profiling.to_csv('customer_profiling_results.csv', index=False)
print("Saved as CSV")

Saved as CSV
